In [1]:
import numpy as np
import pickle
import string 

In [2]:
import numpy as np
import os

def split():
    titles = []
    texts = []
    root = 'Part1'
    
    #dirr = 'Part1/awards_1990/awd_1990_00/'
    dirs = os.listdir('Part1/awards_1990/awd_1990_00/')

    for filename in dirs[1:]:
    #iter = 0
    #print(dirs[1])
            
                #print(iter)
                #iter += 1
                #print(dirs[1:])
                #filename = 'Part1/awards_1990/awd_1990_00/a9000006.txt'
        f = open('Part1/awards_1990/awd_1990_00/' + str(filename))
        addTitle = False
        addTexts = False
        title = []
        text = []
        for word in f.read().split():
            if (word == "Title"):
                addTitle = True
                continue

            if (word == "Type"):
                addTitle = False

#             if (addTexts == True and word == "\n"):
#                 addTexts = False
#                 break


            if (word == "Abstract"):
                addTexts = True
                continue

            if(addTitle == True):
                title.append(word)

            if(addTexts == True):
                text.append(word)

        for i in range(len(title)):
            s = title[i]
            table = str.maketrans({key: None for key in string.punctuation})
            new_s = s.translate(table)
            title[i] = new_s
        for i in range(len(text)):
            s = text[i]
            table = str.maketrans({key: None for key in string.punctuation})
            new_s = s.translate(table)
            text[i] = new_s

        title = ' '.join(title)
        text =' '.join(text)
        titles.append(title)
        texts.append(text)

#     f=open("titles.txt", 'w')
#     for i in titles:
#         f.write(i)
#         f.write(' ')

#     t=open("texts.txt", 'w')
#     for i in texts:
#         t.write(i)
#         t.write(' ')

#     f.close()
#     t.close()
    return titles, texts


In [3]:
titles, texts = split()


In [4]:
print(len(titles))

379


In [5]:
from collections import Counter
from itertools import chain
def get_vocab(lst):
    vocabcount = Counter(w for txt in lst for w in txt.split())
    vocab = map(lambda x: x[0], sorted(vocabcount.items(), key=lambda x: -x[1]))
    return list(vocab), vocabcount

In [7]:
titles[0]

' CRB Genetic Diversity of Endangered Populations of Mysticete Whales Mitochondrial DNA and Historical Demography'

In [8]:
vocab, vocabcount = get_vocab(titles+texts)

In [10]:
len(vocabcount)

9073

In [31]:
from keras.models import Sequential
from keras.engine.topology import Input
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.layers import merge, SpatialDropout1D
from keras.callbacks import TensorBoard
from keras import Model
import keras
import random

In [20]:
vocab_size = 9073
src_txt_length = 379
sum_txt_length = 379
# encoder input model
inputs = Input(shape=(src_txt_length,))
encoder1 = Embedding(vocab_size, 128)(inputs)
encoder2 = LSTM(128)(encoder1)
encoder3 = RepeatVector(sum_txt_length)(encoder2)
# decoder output model
decoder1 = LSTM(128, return_sequences=True)(encoder3)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# tie it together
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [23]:
batch_size = 10
seed = 0
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, 9223372036854775807)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [6]:
FN = 'train'
FN0 = 'vocabulary-embedding'
FN1 = 'train'

import os
import keras
keras.__version__
with open('%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

Using TensorFlow backend.


In [7]:
nflips = 10
traingen = gen(X, Y, batch_size=batch_size, nflips=nflips, model=model)

NameError: name 'gen' is not defined

In [ ]:
len(Y)

In [ ]:
model.fit(X, Y)